In [2]:
#pip install fuzzywuzzy

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import os

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
houseprices = pd.read_csv('house-price-2011.csv')
community = pd.read_excel('community.xlsx')
crime = pd.read_csv('crime.csv')

In [4]:
#DATA PREPROCESSING

#Fill in all empty entries so they return a string
houseprices = houseprices.fillna('')
community = community.fillna('')
crime = crime.fillna('')

In [23]:
#Column Renaming
crime.rename(columns={'Intermediate geography name':'Area'}, inplace=True)
houseprices.rename(columns={'LHF':'Area'}, inplace=True)
houseprices.rename(columns={'STNAME':'Address'}, inplace=True)

In [24]:
#Keyword Filtering
crime['Area'] = crime['Area'].str.replace('Glasgow', '')
houseprices['Area'] = houseprices['Area'].str.replace('and', '')
community['Address'] = community['Address'].str.replace(r"[^a-zA-Z ]+", " ").str.strip()
community['Address'] = community['Address'].str.replace('\d+', '')
community['Address'] = community['Address'].str.replace('Glasgow', '')
community['Address'] = community['Address'].str.replace('Grd', '')
community['Address'] = community['Address'].str.replace('at', '')

In [25]:
#Set all entries to lower case to ensure better match rate
houseprices['Address'] = houseprices['Address'].str.lower()
houseprices['Area'] = houseprices['Area'].str.lower()
crime['Area'] = crime['Area'].str.lower()
community['Address'] = community['Address'].str.lower()

In [26]:
community.drop_duplicates(subset ="Address", 
                     keep = False, inplace = True) 


In [27]:
community

,UPRN,Cost Centre,Propman Estate Code,Ward,Client Name,Description,Address,Property Type
0,,10190,50000010190,10 - Anderston/City,Cat 5-100% owned Common Good,City Halls,candleriggs,Hall
1,906700221993,9220,50000009220,05 - Govan,Cat 5-100% owned Common Good,120 Woodville Street,woodville street,Ground
2,,9029,50000009029,09 - Calton,Cat 5-100% owned Common Good,234 London Road,london road,Ground
3,,5002,50000005002,10 - Anderston/City,Cat 5-100% owned Common Good,Virginia Street (Servitude),virginia street servitude,Ground
9,,501,50000000501,10 - Anderston/City,Cat 5-100% owned Common Good,254-290 Sauchiehall Street,sauchiehall street,Mixed Use
34,,116978,50000660,21 - North East,GCC Dev & Regeneration Service,Rear of 54 Quarrywood Rd,rear of quarrywood rd,Ground
35,,44410,50000640,Outside Glasgow Boundary,GCC Education Services,Nerston Res Sch - Grazings,rd g pd,Other
36,,8229,50000620,05 - Govan,GCC Dev & Regeneration Service,92 Copland Rd,copland rd,Ground
37,,1135,50000600,16 - Canal,GCC Dev & Regeneration Service,Ashfield St - Bardowie St,bardowie st g hj,Ground
38,906700013770,37056,50000580,14 - Drumchapel/Annieslan,CPG Non Tncy/Street Mkts/Other,Blairdardie Pavilion,blairdardie pavilion blairdardie rd g jp,Other


In [28]:
houseprices

,CLASS,STNO,STnu,FLATPOSN,Address,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,QUARTER_(CALENDAR),ACTUAL PRICE AT POINT OF SALE,RPI,DEFLATOR,PRICE CONSTANT AT July 2013,ORIGINOFBUY,OMIT OR USE,NEWBUILD OR RESALE,Area
0,R,106,,1/R,calder st,G42 7RB,1,2011,2010/2011,2011-01-01,2011 Q1,35000,229,1.09039,38163.8,,Use,RESALE,pollokshields southside central
1,R,462,,G/L,caledonia rd,G5 0LG,1,2011,2010/2011,2011-01-01,2011 Q1,70000,229,1.09039,76327.5,FORFAR,Use,RESALE,pollokshields southside central
2,R,,,,richmond gate,G5,1,2011,2010/2011,2011-01-01,2011 Q1,185000,229,1.09039,201723,GLASGOW,Use,NEW,pollokshields southside central
3,R,490,,15-Apr,"argyle st,the argyle buil",G2 8AJ,1,2011,2010/2011,2011-01-01,2011 Q1,165000,229,1.09039,179915,LARBERT,Use,NEW,central west
4,R,490,,06-Feb,"argyle st,the argyle buil",G2 8AJ,1,2011,2010/2011,2011-01-01,2011 Q1,143500,229,1.09039,156471,LUXEMBOURG,Use,NEW,central west
5,R,490,,08-Apr,"argyle st,the argyle buil",G2 8AJ,1,2011,2010/2011,2011-01-01,2011 Q1,156000,229,1.09039,170101,SKELMORLIE,Use,NEW,central west
6,R,109,,,barfillan dr,G52 1BD,1,2011,2010/2011,2011-01-01,2011 Q1,120000,229,1.09039,130847,GLASGOW,Use,NEW,govan craigton
7,R,55,,,"belvidere ave,belvidere vi",G31 4PA,1,2011,2010/2011,2011-01-01,2011 Q1,115000,229,1.09039,125395,MID CALDER,Use,NEW,east centre calton
8,R,4,,,"belvidere ter,belvidere v",G31 4PD,1,2011,2010/2011,2011-01-01,2011 Q1,139995,229,1.09039,152650,FALKIRK,Use,NEW,east centre calton
9,R,33,,,"blackhill dr,the grange",G23 5NH,1,2011,2010/2011,2011-01-01,2011 Q1,250000,229,1.09039,272598,GLASGOW,Use,NEW,maryhill kelvin canal


In [29]:
#Merge crime and houseprices columns
housing_and_crime = houseprices.merge(crime, 
                        left_on='Area', 
                        right_on='Area', 
                        how='outer', 
                        suffixes=["","_Area"] #since they are the same name, we need a suffix
                       )
housing_and_crime

,CLASS,STNO,STnu,FLATPOSN,Address,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,...,Area,Datazone,Multi-member ward name,2011 Scottish Parliamentary Constituencies,crime_deprivation_rank_2006,crime_deprivation_rank_2009,crime_deprivation_rank_2012,crime_local_rank_2006,crime_local_rank_2009,crime_local_rank_2012
0,R,106,,1/R,calder st,G42 7RB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R,462,,G/L,caledonia rd,G5 0LG,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R,,,,richmond gate,G5,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R,10,,02-Mar,"haughview ter,richmond ga",G5 0HB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R,152,,,"newhouse rd,crown gdns",G42 0EB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R,164,,,"newhouse rd,crown gdns",G42 0EB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,R,72-76,,01-Feb,queens dr,G42 8BW,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,R,7,,,"fauldhouse way,richmond gate",G5 0JB,2,2011.0,2010/2011,2011-02-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,R,31,,,"fauldhouse way,richmond gate",G5 0JB,3,2011.0,2010/2011,2011-03-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,R,10,,03-Feb,"haughview ter,richmond ga",G5 0HB,3,2011.0,2010/2011,2011-03-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
matched_crime_areas = housing_and_crime[housing_and_crime.Address.notnull() & housing_and_crime.Datazone.notnull()]
matched_crime_areas

,CLASS,STNO,STnu,FLATPOSN,Address,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,...,Area,Datazone,Multi-member ward name,2011 Scottish Parliamentary Constituencies,crime_deprivation_rank_2006,crime_deprivation_rank_2009,crime_deprivation_rank_2012,crime_local_rank_2006,crime_local_rank_2009,crime_local_rank_2012
4145,R,79,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003572,Springburn,Glasgow Springburn,322.0,288.0,545.0,66.0,51.0,106.0
4146,R,79,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003644,Springburn,Glasgow Springburn,1392.0,790.0,655.0,285.0,136.0,133.0
4147,R,79,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003606,Springburn,Glasgow Springburn,1730.0,2659.0,2250.0,335.0,427.0,415.0
4148,R,79,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003573,Springburn,Glasgow Springburn,3193.0,4012.0,2326.0,527.0,587.0,424.0
4149,R,79,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003565,Springburn,Glasgow Springburn,1888.0,1865.0,2865.0,367.0,334.0,481.0
4150,R,81,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003572,Springburn,Glasgow Springburn,322.0,288.0,545.0,66.0,51.0,106.0
4151,R,81,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003644,Springburn,Glasgow Springburn,1392.0,790.0,655.0,285.0,136.0,133.0
4152,R,81,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003606,Springburn,Glasgow Springburn,1730.0,2659.0,2250.0,335.0,427.0,415.0
4153,R,81,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003573,Springburn,Glasgow Springburn,3193.0,4012.0,2326.0,527.0,587.0,424.0
4154,R,81,,,"cortmalaw cres,wallacefie",G33 1TD,1,2011.0,2010/2011,2011-01-01,...,springburn,S01003565,Springburn,Glasgow Springburn,1888.0,1865.0,2865.0,367.0,334.0,481.0


In [31]:
housing_and_crime = housing_and_crime[housing_and_crime.Address.isnull() | housing_and_crime.Datazone.isnull()]
housing_and_crime

,CLASS,STNO,STnu,FLATPOSN,Address,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,...,Area,Datazone,Multi-member ward name,2011 Scottish Parliamentary Constituencies,crime_deprivation_rank_2006,crime_deprivation_rank_2009,crime_deprivation_rank_2012,crime_local_rank_2006,crime_local_rank_2009,crime_local_rank_2012
0,R,106,,1/R,calder st,G42 7RB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R,462,,G/L,caledonia rd,G5 0LG,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R,,,,richmond gate,G5,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R,10,,02-Mar,"haughview ter,richmond ga",G5 0HB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R,152,,,"newhouse rd,crown gdns",G42 0EB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R,164,,,"newhouse rd,crown gdns",G42 0EB,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,R,72-76,,01-Feb,queens dr,G42 8BW,1,2011.0,2010/2011,2011-01-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,R,7,,,"fauldhouse way,richmond gate",G5 0JB,2,2011.0,2010/2011,2011-02-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,R,31,,,"fauldhouse way,richmond gate",G5 0JB,3,2011.0,2010/2011,2011-03-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,R,10,,03-Feb,"haughview ter,richmond ga",G5 0HB,3,2011.0,2010/2011,2011-03-01,...,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
crime = crime[crime.crime_local_rank_2012 < 100][['Area','crime_local_rank_2012']]

crime.reset_index(inplace=True,drop='index')
crime.head(100)

,Area,crime_local_rank_2012
0,city centre west,1
1,city centre west,2
2,parkhead west and barrowfield,3
3,city centre east,4
4,city centre west,5
5,drumoyne and shieldhall,6
6,laurieston and tradeston,7
7,drumry east,8
8,"calton, gallowgate and bridgeton",9
9,battlefield,10


In [33]:
housing_and_communities = houseprices.merge(community, 
                        left_on='Address', 
                        right_on='Address', 
                        how='outer', 
                        suffixes=["","_Address"] #since they are the same name, we need a suffix
                                           )         
housing_and_communities

,CLASS,STNO,STnu,FLATPOSN,Address,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,...,OMIT OR USE,NEWBUILD OR RESALE,Area,UPRN,Cost Centre,Propman Estate Code,Ward,Client Name,Description,Property Type
0,R,106,,1/R,calder st,G42 7RB,1,2011.0,2010/2011,2011-01-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R,50,,3/2L,calder st,G42 7RU,1,2011.0,2010/2011,2011-01-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R,50,,2/E,calder st,G42 7RU,2,2011.0,2010/2011,2011-02-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R,118,,G/L,calder st,G42 7RB,2,2011.0,2010/2011,2011-02-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R,106,,1/R,calder st,G42 7RB,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R,112,,3/L,calder st,G42 7RB,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,R,26,,G/R,calder st,G42 7RU,4,2011.0,2011/2012,2011-04-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,R,345,,1/L,calder st,G42 7NT,5,2011.0,2011/2012,2011-05-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,R,214,,1/M,calder st,G42 7PE,9,2011.0,2011/2012,2011-09-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,R,214,,1/M,calder st,G42 7PE,11,2011.0,2011/2012,2011-11-01,...,Use,RESALE,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
matched_communities = housing_and_communities[housing_and_communities.CLASS.notnull() & housing_and_communities.Description.notnull()]
matched_communities

,CLASS,STNO,STnu,FLATPOSN,Address,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,...,OMIT OR USE,NEWBUILD OR RESALE,Area,UPRN,Cost Centre,Propman Estate Code,Ward,Client Name,Description,Property Type
3017,R,83,,01-Apr,candleriggs,G1 1LF,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,central west,,10190,5.000001e+10,10 - Anderston/City,Cat 5-100% owned Common Good,City Halls,Hall
3018,R,83C,,F5,candleriggs,G1 1LF,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,central west,,10190,5.000001e+10,10 - Anderston/City,Cat 5-100% owned Common Good,City Halls,Hall
3019,R,79,,01-Mar,candleriggs,G1 1NP,4,2011.0,2011/2012,2011-04-01,...,Use,RESALE,central west,,10190,5.000001e+10,10 - Anderston/City,Cat 5-100% owned Common Good,City Halls,Hall
3020,R,83,,01-Feb,candleriggs,G1 1LF,8,2011.0,2011/2012,2011-08-01,...,Use,RESALE,central west,,10190,5.000001e+10,10 - Anderston/City,Cat 5-100% owned Common Good,City Halls,Hall
3021,R,83,,01-Jan,candleriggs,G1 1LF,11,2011.0,2011/2012,2011-11-01,...,Use,RESALE,central west,,10190,5.000001e+10,10 - Anderston/City,Cat 5-100% owned Common Good,City Halls,Hall


In [35]:
housing_and_communities = housing_and_communities[housing_and_communities.CLASS.isnull() | housing_and_communities.Description.isnull()]
housing_and_communities

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,CLASS,STNO,STnu,FLATPOSN,Address,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,...,OMIT OR USE,NEWBUILD OR RESALE,Area,UPRN,Cost Centre,Propman Estate Code,Ward,Client Name,Description,Property Type
418,R,76,,,"tillycairn rd,the glen",G33 5EH,1,2011.0,2010/2011,2011-01-01,...,Use,NEW,baillieston shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,R,41,,,"blackhill gdns,the grange",G23 5NE,2,2011.0,2010/2011,2011-02-01,...,Use,NEW,maryhill kelvin canal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,R,8,,G/1,"haggs gate,the green",G41 4BB,2,2011.0,2010/2011,2011-02-01,...,Use,NEW,greater pollok newls auldburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622,R,46,,02-Jan,elderpark st,G51 3SU,3,2011.0,2010/2011,2011-03-01,...,Use,NEW,govan craigton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
660,R,17,,,"newhouse dr,crown gdns",G42 0EE,3,2011.0,2010/2011,2011-03-01,...,Use,NEW,pollokshields southside central,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,R,56,,F3,"buchanan st,the ctyard",G69 6DY,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,baillieston shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN
705,R,15,,,haig dr,G69 7JW,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,baillieston shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN
706,R,16,,,halton gdns,G69 7LD,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,baillieston shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,R,17,,,langton gdns,G69 7LE,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,baillieston shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN
785,R,11,,,rhindmuir pl,G69 6HF,3,2011.0,2010/2011,2011-03-01,...,Use,RESALE,baillieston shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [37]:
dict_list = []
for name in houseprices.Area:
    match = match_name(name, crime.Area, 50)
    # New dict for storing data
    dict_ = {}
    dict_.update({"crime_area" : name})
    dict_.update({"area" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table = merge_table.sort_values(by=['score'], ascending=False)
merge_table

,area,crime_area,score
6286,city centre west,central west,69
5554,city centre west,central west,69
1744,city centre west,central west,69
5141,city centre west,central west,69
3579,city centre west,central west,69
1739,city centre west,central west,69
3581,city centre west,central west,69
5226,city centre west,central west,69
5555,city centre west,central west,69
1734,city centre west,central west,69


In [38]:
dict_list2 = []
for name in houseprices.Address:
    match = match_name(name, community.Address, 50)
    
    # New dict for storing data
    dict2_ = {}
    dict2_.update({"community_address" : name})
    dict2_.update({"address" : match[0]})
    dict2_.update({"score" : match[1]})
    dict_list2.append(dict2_)
    
merge_table2 = pd.DataFrame(dict_list2)
# Display results
merge_table2 = merge_table2.sort_values(by=['score'], ascending=False)
merge_table2

,address,community_address,score
3693,candleriggs,candleriggs,100
1522,candleriggs,candleriggs,100
5951,candleriggs,candleriggs,100
953,candleriggs,candleriggs,100
954,candleriggs,candleriggs,100
4014,mosspark boulevard,mosspark boulevard,95
4013,mosspark boulevard,mosspark boulevard,95
515,blairdardie rd,blairdardie rd,93
2308,blairdardie rd,blairdardie rd,93
3160,blairdardie rd,blairdardie rd,93
